In [4]:
%%capture
!pip install numpy 
!pip install pandas 
!pip install matplotlib
!pip install tensorflow
!pip install scikit-learn
!pip install ipywidgets


Base.Meta.ParseError: ParseError:
# Error @ c:\Users\param\OneDrive\Desktop\COMPP\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:1:1
%%capture
╙ ── not a unary operator

In [5]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import ipywidgets as widgets
from IPython.display import display, HTML

# Load data
data = pd.read_csv("Vegetables.csv")

# Preprocess data
data = data.rename(columns={'Price (Rs./Quintal)': 'Price'})
data.dropna(inplace=True)

# Dropdown list for vegetables
vegetable_list = data['Commodity'].unique()

# Add CSS styling
style = """
<style>
    .widget-box {
        background-color: #f9f9f9;
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        margin: auto;
        width: 60%;
        text-align: center;
    }
    .output-area {
        background-color: #e8f4fa;
        padding: 20px;
        border-radius: 10px;
        margin-top: 10px;
    }
</style>
"""
display(HTML(style))

# Create input widgets
location_input = widgets.Text(
    value='RR Nagar',
    description='Location:',
    placeholder='Enter your location',
    style={'description_width': 'initial'}
)

vegetable_input = widgets.Dropdown(
    options=vegetable_list,
    description='Vegetable:',
    style={'description_width': 'initial'}
)

# Button to trigger prediction
predict_button = widgets.Button(
    description="Predict Prices",
    button_style='primary'
)
output = widgets.Output()

# Align widgets in a VBox
input_widgets = widgets.VBox([location_input, vegetable_input, predict_button])
input_widgets.add_class("widget-box")

# Function to handle predictions
def predict_prices(b):
    with output:
        output.clear_output()  # Clear previous output
        
        # Get user inputs
        selected_location = location_input.value.strip().title()
        selected_vegetable = vegetable_input.value.strip().title()
        
        # Filter data for selected vegetable
        vegetable_data = data[data['Commodity'] == selected_vegetable]
        
        if vegetable_data.empty:
            print(f"No data available for {selected_vegetable}.")
            return
        
        # Preprocess data for model input
        scaler = MinMaxScaler(feature_range=(0, 1))
        vegetable_prices = vegetable_data.Price.values.reshape(-1, 1)
        scaled_prices = scaler.fit_transform(vegetable_prices)
        
        # Use the last 10 days of data for predictions
        last_10_days = scaled_prices[-10:]
        last_10_days = last_10_days.reshape(1, 10, 1)
        
        # Load pre-trained model and make predictions
        model = load_model('Comodity Price Predictor.keras')
        predicted_prices = []
        
        for _ in range(3):  # Predict for 3 days
            predicted_price_scaled = model.predict(last_10_days)
            predicted_price = scaler.inverse_transform(predicted_price_scaled)[0][0]
            predicted_prices.append(predicted_price)
            
            # Update the input with the latest prediction
            last_10_days = np.append(last_10_days[0, 1:], predicted_price_scaled).reshape(1, 10, 1)
        
        # Convert to price per kg
        predicted_prices_kg = [price / 100 for price in predicted_prices]
        
        # Display results
        print(f"Location: {selected_location}")
        print(f"Vegetable: {selected_vegetable}")
        for i, price in enumerate(predicted_prices_kg, 1):
            print(f"Predicted price for day {i} (per kg): ₹{price:.2f}")
        
        # Plot results

# Bind button click to prediction function
predict_button.on_click(predict_prices)

# Display styled widgets and output area
display(widgets.VBox([input_widgets, output]))


ArgumentError: ArgumentError: Package numpy not found in current path.
- Run `import Pkg; Pkg.add("numpy")` to install the numpy package.